### CNN 모델로 챗봇 문답 데이터 감정 분류 모델 구현

In [1]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [2]:
# 데이터 읽어오기
train_file = '/content/ChatbotData.csv'
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

pandas의 read_csv()함수를 사용하여 csv 파일 읽기<br>
CNN 모델 학습에 필요한 것들을 리스트로 저장
- Q(질문)
- label(감정)


In [3]:
# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index

MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

features의 문장들을 sequence() 함수를 이용해 단어 시퀀스 생성<br>
생성된 시퀀스를 corpus 리스트에 저장하고 texts_to_sequence() 함수를 이용하여 시퀀스 번호로 변환<br>
변환된 시퀀스 번호를 이용하여 단어 임베딩 벡터를 만들고 크기 조정<br>
패딩padding 처리

In [4]:
# 학습용, 검증용, 테스트셋 생성 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

데이터를 랜덤으로 섞은 후 비율에 맞추어 분리

In [5]:
# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1 # 전체 단어 수

dropout_prob: dropout 생성률 <br>
EMB_SIZE: 임베딩 결과로 나올 밀집 벡터의 크기

In [6]:
from tensorflow.python.ops.gen_nn_ops import Conv2D
# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,))
embeddding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embeddding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)
conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)
conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

In [7]:
# 3, 4, 5-gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden)
predictions = Dense(3, activation=tf.nn.softmax)(logits)

완전 완결 계층 구현<br>
Dense()를 이용하여 128개의 출력 노드를 가지고, relu 활성화 함수를 사용하는 계층 생성<br>
Dense 계층은 이전 계층에서 합성곱 연산과 맥스 풀링으로 나온 3개의 특징맵 데이터를 입력으로 받음<br>
가장 큰 점수를 가진 노드의 위치가 XNN 모델이 예측한 class가 된다.<br>

마지막 출력 노드로 정의한 logits에서 나온 점수를 소프트맥스 계층을 통해 확률 계산<br>
클래스 분류 모델 classification problem을 학습할 때 주오 **손실값loss**를 계산하는 함수로<br>
**sparse_categorical_crossentropy()**를 사용하고<br>
크로스엔트로피 cross_entropy 계산을 위해 **확률값**이 필요해 **소프트맥스 계층**이 필요


In [8]:
# 모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Model(): 케라스 모델을 생성하는 함수<br>
input_layer(입력 계층), predictions(출력 계층) 사용<br>
모델 정의 후 model.compile() 함수를 통해 컴파일<br>
**최적화**를 위하여 adam, **손실함수**에는 sparse_categorical_crossentropy<br>
정확도 확인을 위해 metrics의 accuracy 사용

In [9]:
# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/5
414/414 [==============================] - 39s 83ms/step - loss: 0.8818 - accuracy: 0.5790 - val_loss: 0.5129 - val_accuracy: 0.8139
Epoch 2/5
414/414 [==============================] - 23s 55ms/step - loss: 0.4928 - accuracy: 0.8165 - val_loss: 0.2684 - val_accuracy: 0.9234
Epoch 3/5
414/414 [==============================] - 24s 57ms/step - loss: 0.3056 - accuracy: 0.8996 - val_loss: 0.1615 - val_accuracy: 0.9522
Epoch 4/5
414/414 [==============================] - 23s 54ms/step - loss: 0.1977 - accuracy: 0.9380 - val_loss: 0.0970 - val_accuracy: 0.9712
Epoch 5/5
414/414 [==============================] - 17s 40ms/step - loss: 0.1349 - accuracy: 0.9584 - val_loss: 0.0670 - val_accuracy: 0.9788


첫 번째 인자: 학습용 데이터셋<br>
두 번째 인자 validation_data: 검증용 데이터셋<br>
세 번째 인자 epochs: 학습 반복 횟수<br>
네 번째 인자 verbose: 1인 경우 학습 시 진행 과정을 상세히 보여준다.<br>

In [10]:
# 모델 평가(테스트 데이터 사용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy*100))

60/60 [==============================] - 0s 5ms/step - loss: 0.0722 - accuracy: 0.9772
Accuracy: 97.715735


evaluate() 함수를 이용하여 성능 평가<br>
인자: 테스트용 데이터셋

In [11]:
# 모델 저장
model.save('cnn_model.h5')

___

### 챗봇 문답 데이터 감정 분류 모델 사용

In [12]:
# 필요한 모듈 임포트
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

In [13]:
# 데이터 읽어오기
train_file = "/content/ChatbotData.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

labels: 예측 결과와 실제 분류값 비교

In [14]:
# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)

MAC_SEQ_LEN = 15
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAC_SEQ_LEN, padding='post')

질문을 하나씩 꺼내 text_to_word_sequence()를 통해 단어 시퀀스로 만든 후 corpus 리스트에 저장<br>
문장 내 모든 단어를 가지고 texts_to_sequences()를 통해 시퀀스 번호로 변환<br>
벡터의 크기를 맞추기 위하여 pad_sequence()

In [19]:
# 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20)

패딩 처리한 시퀀스(padded_seqs) 벡터 리스트와 감정 리스트를 데이터셋 객체로 만들기<br>
랜덤으로 섞은 후 테스트용 데이터셋을 2000개 뽑아 20개씩 배치 처리

In [21]:
# 감정 분류 CNN 모델 불러오기
model = load_model('cnn_model.h5')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 15, 128)      1715072     ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 15, 128)      0           ['embedding[0][0]']              
                                                                                                  
 conv1d (Conv1D)                (None, 13, 128)      49280       ['dropout[0][0]']                
                                                                                              

모델을 불러와서 학습하고 성능 평가

In [22]:
# 테스트용 데ㅣㅇ터셋의 10212번째 데이터 출력
print("단어 시퀀스: ", corpus[10212])
print("단어 인덱스 시퀀스: ", padded_seqs[10212])
print("문장 분류(정답): ", labels[10212])

단어 시퀀스:  ['썸', '타는', '여자가', '남사친', '만나러', '간다는데', '뭐라', '해']
단어 인덱스 시퀀스:  [   13    61   127  4320  1333 12162   856    31     0     0     0     0
     0     0     0]
문장 분류(정답):  2


감정 예측

In [24]:
# 테스트용 데이터셋의 10212번째 데이터 감정 예측
picks = [10212]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print("감정 예측 점수: ", predict)
print("감정 예측 클래스: ", predict_class.numpy())

감정 예측 점수:  [[1.0250853e-05 3.0599933e-06 9.9998665e-01]]
감정 예측 클래스:  [2]


예측 점수 반환